In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - fit_dens.ipynb
# AUTHOR - James Lane
# PROJECT - ges-mass
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Fit density profiles to real APOGEE data
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import os, sys, pdb, time, tqdm, warnings, multiprocessing, copy, dill as pickle
import numpy as np

# Matplotlib and plotting 
import matplotlib
import matplotlib.pyplot as plt
import corner

# mwdust, isodist stuff, galpy stuff
import mwdust
from isodist import FEH2Z, Z2FEH
from galpy.util import coords

# Fitting, optimization and statistics
import emcee
import scipy.optimize

# Project specific
sys.path.insert(0,'../../src/')
from ges_mass import mass as pmass
from ges_mass import densprofiles as pdens
from ges_mass import iso as piso
from ges_mass import util as putil
from ges_mass import plot as pplot

### Scale parameters
ro = 8.275
vo = 220
zo = 0.0208 # Bennett+ 2019

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
## Keywords
cdict = putil.load_config_to_dict()
keywords = ['BASE_DIR','APOGEE_DR','APOGEE_RESULTS_VERS','GAIA_DR','NDMOD',
            'DMOD_MIN','DMOD_MAX','LOGG_MIN','LOGG_MAX','FEH_MIN','FEH_MAX',
            'NPROCS']
base_dir,apogee_dr,apogee_results_vers,gaia_dr,ndmod,dmod_min,dmod_max,\
    logg_min,logg_max,feh_min,feh_max,nprocs\
    = putil.parse_config_dict(cdict,keywords)
logg_range = [logg_min,logg_max]
feh_range = [feh_min,feh_max]

In [ ]:
## Pathing
data_dir = base_dir+'data/'
version_dir = 'apogee_'+apogee_dr+'_'+apogee_results_vers+'_gaia_'+gaia_dr+'/'
ga_dir = data_dir+'gaia_apogee/'+version_dir
gap_dir = data_dir+'gaia_apogee_processed/'+version_dir
ksf_dir = data_dir+'ksf/'+version_dir

In [ ]:
## Filenames
apogee_SF_filename = ga_dir+'apogee_SF.dat'
apogee_effSF_filename = ga_dir+\
    'apogee_effSF_grid_inclArea.dat'
apogee_effSF_mask_filename = ga_dir+\
    'apogee_effSF_grid_mask.npy'
iso_grid_filename = ga_dir+'iso_grid.npy'
clean_kinematics_filename = gap_dir+'clean_kinematics.npy'

In [ ]:
## Loading

# selection function
with open(apogee_SF_filename, 'rb') as f:
    print('\nLoading APOGEE sel. func. from '+apogee_SF_filename)
    apogee_SF = pickle.load(f)
# with open(apogee_effSF_filename,'rb') as f:
#     print('\nLoading APOGEE eff. sel. func. from '+apogee_effSF_filename)
#     apogee_effSF_grid_inclArea = pickle.load(f)
print('\nLoading APOGEE eff. sel. func. mask from '+apogee_effSF_mask_filename)
apogee_effSF_mask = np.load(apogee_effSF_mask_filename)

# Data
with open(clean_kinematics_filename,'rb') as f:
    print('Loading cleaned kinematics from '+clean_kinematics_filename)
    clean_kinematics = pickle.load(f)

_,allstar_nomask,orbs_nomask,_,_,_ = clean_kinematics

# Dust map
dmap = mwdust.Combined19(filter='2MASS H') # dustmap from mwdust, use most recent

# Isochrone
print('\nLoading isochrone grid from '+iso_grid_filename)
iso_grid = np.load(iso_grid_filename)

In [ ]:
## Effective selection function grid
# Distance modulus grid
dmods,ds = putil.make_dmod_grid(ndmod,dmod_min,dmod_max)

# Grid of positions in the APOGEE effective selection function grid
Rgrid,phigrid,zgrid = pmass.Rphizgrid(apogee_SF,dmods,ro=ro,zo=zo)

# # Apply the effective selection function grid mask
# apof = apogee_effSF_grid_inclArea[apogee_effSF_mask]
Rgrid = Rgrid[apogee_effSF_mask]
phigrid = phigrid[apogee_effSF_mask]
zgrid = zgrid[apogee_effSF_mask]

# # Include the distance modulus Jacobian
# Jac_dmod = ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])
# Jac_rad = (np.pi/180.)**2.
# apof = apof * Jac_dmod * Jac_rad

# JKmins
jkmins = np.array([apogee_SF.JKmin(apogee_SF._locations[i]) \
                   for i in range(len(apogee_SF._locations))])
jkmins = jkmins[apogee_effSF_mask]

### Fit e-Lz

In [ ]:
# Keywords
selec_spaces = ['eLz']
selec_spaces_suffix = '-'.join(selec_spaces)

apogee_keffSF_filename = ksf_dir+'apogee_keffSF_grid_inclArea_'+\
                                 selec_spaces_suffix+'.dat'

with open(apogee_keffSF_filename,'rb') as f:
    print('\nLoading APOGEE kin. eff. sel. func. from '+apogee_keffSF_filename)
    apogee_keffSF_grid_inclArea = pickle.load(f)

# Apply the effective selection function grid mask and Jacobians
apokf = apogee_keffSF_grid_inclArea[apogee_effSF_mask]
Jac_dmod = ds**3.*np.log(10)/5.*(dmods[1]-dmods[0])
Jac_rad = (np.pi/180.)**2.
apokf = apokf * Jac_dmod * Jac_rad

#### Line masking

In [ ]:
# GE mask type
dmask_type = 'poly' # lines or poly

# Mask out GE stars
GE_apogee_IDs = np.load(gap_dir+'hb_apogee_ids_'+selec_spaces_suffix+\
                                '_'+dmask_type+'.npy')
GE_mask = np.in1d( allstar_nomask['APOGEE_ID'].astype(str), GE_apogee_IDs )
orbs_GE = orbs_nomask[GE_mask]
allstar_GE = allstar_nomask[GE_mask]

# All fields pass the null test?
assert np.all( ~np.all(apokf < 1e-9, axis=1) )

# Obervational masking
data_mask = (allstar_GE['FE_H'] > feh_min) &\
            (allstar_GE['FE_H'] < feh_max) &\
            (allstar_GE['LOGG'] > logg_min) &\
            (allstar_GE['LOGG'] < logg_max)
orbs = orbs_GE[data_mask]
allstas = allstar_GE[data_mask]
mrpz = np.array([orbs.R(use_physical=True).value,
                 orbs.phi(use_physical=True).value,
                 orbs.z(use_physical=True).value]).T
print(str(len(orbs))+' stars') 

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
densfunc_nodisk = pdens.triaxial_single_angle_zvecpa
# mcmc_labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\phi$', r'PA']
mcmc_labels = [r'$\alpha$', r'$p$', r'$q$', r'$\eta$', r'$\theta$', 
               r'$\phi$', r'$f_{disc}$']
# mcmc_labels = [r'$\alpha$', r'$\beta$', r'$p$', r'$q$', r'$\eta$', r'$\theta$', 
#                r'$\phi$', r'$f_{disc}$']

init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1])
# init = np.array([2.0, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1])


nwalkers, nit, ncut = 100, 3000, 1000
n_mass = 2000
r_min,r_max = 2.0, 70.0
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apokf)

# Do MLE and MCMC
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=init, nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)

fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    labels=mcmc_labels)

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples[:,:-1], 
    densfunc=densfunc_nodisk, n_star=len(orbs), effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso_grid, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    labels=[r'Mass [$10^{8}$ Msol]'])

fig.show()

np.save('samples_single_angle.npy',samples)
np.save('masses_single_angle.npy',masses)

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_cutoff_zvecpa_plusexpdisk
# densfunc_nodisk = pdens.triaxial_single_cutoff_zvecpa
# mcmc_labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\phi$', r'PA']
mcmc_labels = [r'$\alpha$', r'$\beta$', r'$p$', r'$q$', r'$\eta$', r'$\theta$', 
               r'$\phi$', r'$f_{disc}$']
# mcmc_labels = [r'$\alpha$', r'$\beta$', r'$p$', r'$q$', r'$\eta$', r'$\theta$', 
#                r'$\phi$', r'$f_{disc}$']

# init = np.array([2.0, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1])
init = np.array([2.0, 0.1, 0.5, 0.5, 0.5, 0.5, 0.5, 0.1])


nwalkers, nit, ncut = 100, 3000, 1000
n_mass = 2000
r_min,r_max = 2.0, 70.0
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apokf)

# Do MLE and MCMC
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=init, nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)

fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    labels=mcmc_labels)

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples[:,:-1], 
    densfunc=densfunc_nodisk, n_star=len(orbs), effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso_grid, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    labels=[r'Mass [$10^{8}$ Msol]'])

np.save('samples_single_cutoff.npy',samples)
np.save('masses_single_cutoff.npy',masses)

fig.show()

#### Polygon masking

In [ ]:
# GE mask type
dmask_type = 'poly' # lines or poly

# Mask out GE stars
GE_apogee_IDs = np.load(gap_dir+'hb_apogee_ids_'+selec_spaces_suffix+\
                                '_'+dmask_type+'.npy')
GE_mask = np.in1d( allstar_nomask['APOGEE_ID'].astype(str), GE_apogee_IDs )
orbs_GE = orbs_nomask[GE_mask]
allstar_GE = allstar_nomask[GE_mask]

# All fields pass the null test?
assert np.all( ~np.all(apokf < 1e-9, axis=1) )

# Obervational masking
data_mask = (allstar_GE['FE_H'] > feh_min) &\
            (allstar_GE['FE_H'] < feh_max) &\
            (allstar_GE['LOGG'] > logg_min) &\
            (allstar_GE['LOGG'] < logg_max)
orbs = orbs_GE[data_mask]
allstas = allstar_GE[data_mask]
mrpz = np.array([orbs.R(use_physical=True).value,
                 orbs.phi(use_physical=True).value,
                 orbs.z(use_physical=True).value]).T
print(str(len(orbs))+' stars') 

In [ ]:
# Definitions
densfunc = pdens.triaxial_single_angle_zvecpa_plusexpdisk
densfunc_nodisk = pdens.triaxial_single_angle_zvecpa
# mcmc_labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\phi$', r'PA']
mcmc_labels = [r'$\alpha$', r'$p$', r'$q$', r'$\theta$', r'$\phi$', r'PA', 
               r'$f_{disc}$']
# alpha, p, q, eta, theta, pa, fdisc
init = np.array([2.0, 0.9, 0.9, 0.5, 0.5, 0.5, 0.1])
nwalkers, nit, ncut = 25, 200, 100
n_mass = 200
r_min,r_max = 1.0, 70.
Rdata,phidata,zdata = mrpz.T
effsel = copy.deepcopy(apokf)

# Do MLE and MCMC
samples, opt = pmass.fit_dens(densfunc=densfunc, effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], data=[Rdata,phidata,zdata], 
    init=init, nprocs=9, nwalkers=nwalkers, nit=nit, ncut=ncut, 
    MLE_init=True, just_MLE=False)

fig = corner.corner(samples, show_titles=True, quantiles=[0.16,0.5,0.84], 
                    labels=mcmc_labels)

# Calculate mass
masses, facs = pmass.mass_from_density_samples(samples=samples[:,:-1], 
    densfunc=densfunc_nodisk, n_star=len(orbs), effsel=effsel, 
    effsel_grid=[Rgrid,phigrid,zgrid], iso=iso_grid, feh_range=feh_range, 
    logg_range=logg_range, jkmins=jkmins, n_mass=n_mass, 
    mass_int_type='spherical_grid', int_r_range=[r_min,r_max], ro=ro, zo=zo)

fig = corner.corner(masses/1e8, show_titles=True, quantiles=[0.16,0.5,0.84],
                    labels=[r'$Mass [$10^{8}$ Msol]'])

fig.show()